In [1]:
# Python
import pandas as pd
import numpy as np
from fbprophet import Prophet
# Data process
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso, Ridge, MultiTaskLassoCV, MultiTaskElasticNetCV,LassoCV
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import BaggingRegressor, AdaBoostRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
# Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
import plotly.plotly as py
import plotly.graph_objs as go
# Other
import datetime
import pickle
from plotly_utils.utils import get_plotly_fig

%matplotlib inline
pylab.rcParams['figure.figsize'] = 18, 6

In [2]:
def MAPE(y_predict, y_true):
    return np.mean(np.abs(y_predict - y_true) / y_true)

In [3]:
task2_dataset = pickle.load(open('dataset.pkl', 'rb'))
X = task2_dataset['X_train']
X_test = task2_dataset['X_test']
y_train = task2_dataset['y_train']

X_train = X[:'2016-10-15 7:40']
X_valid = X.loc[pd.to_datetime([
    '2016-10-15 17:00',
    '2016-10-15 8:00',
    '2016-10-16 17:00',
    '2016-10-16 8:00',
    '2016-10-17 17:00',
    '2016-10-17 8:00']
)]

full_X = pd.concat([X, X_test])
full_X = full_X.swaplevel(0, 1, 1).sort_index(1)
y_train = y_train.swaplevel(0, 1, 1).sort_index(1)

full_X.columns = ['.'.join(col).strip() for col in full_X.columns.values]
y_train.columns = ['.'.join(col).strip() for col in y_train.columns.values]

In [233]:
full_X = np.log(full_X+1)
y_train = np.log(y_train+1)

In [4]:
BeforeNationalDay = (
    datetime.date(2016, 9, 30),
)

NationalDayFront = (
    datetime.date(2016, 10, 1),
    datetime.date(2016, 10, 2),
    datetime.date(2016, 10, 3),
    datetime.date(2016, 10, 4),
)

NationalDayTail = (
    datetime.date(2016, 10, 5),
    datetime.date(2016, 10, 6),
    datetime.date(2016, 10, 7)
)

WorkingWeekend = (
    datetime.date(2016, 9, 18),
    datetime.date(2016, 10, 8),
    datetime.date(2016, 10, 9)
)


def get_day(date):
    dayofweek = date.dayofweek
    date = date.date()
    if date in BeforeNationalDay:
        return "BeforeNationalDay"
    if date in NationalDayFront:
        return "NationalDayFront(1-4)"
    if date in NationalDayTail:
        return "NationalDayTail(5-7)"
    if date in WorkingWeekend:
        return "WorkingWeekend"
    if dayofweek in (0, 1, 2, 3, 4):
        return "WorkingDay"
    else:
        return "Weekend"


# DayTypeDummies
dayTypeDummies = pd.get_dummies(full_X.index.map(get_day))
dayTypeDummies.index = full_X.index
full_X = full_X.join(dayTypeDummies)

# HourDummies
hours = full_X.index.map(lambda dt: dt.hour)
hourDummies = pd.get_dummies(hours, prefix='Hour')
hourDummies.index = full_X.index
full_X = full_X.join(hourDummies)

weather = pd.read_pickle('../datasets/weather_feature.pkl')
weather = weather.loc[full_X.index].fillna(0)[['Heavy rain', 'Light rain', 'Rain', 'Sunny']].astype(int)
full_X = full_X.join(weather)

In [ ]:
sns.pairplot(full_X)

In [140]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor,AdaBoostRegressor,ExtraTreesRegressor,GradientBoostingRegressor

In [ ]:
model = MultiOutputRegressor(GradientBoostingRegressor(
    n_estimators=100, learning_rate=0.1,     max_depth=2, random_state=0, loss='ls'))
X_train_value = full_X.loc[X_train.index]
y_train_value = y_train.loc[X_train.index].iloc[:, 6:12]

X_valid_value = full_X.loc[X_valid.index]
y_valid_value = y_train.loc[X_valid.index].iloc[:, 6:12]
model.fit(X_train_value, y_train_value)


np.mean(np.abs(model.predict(X_valid_value).astype(
    int) - y_valid_value.values) / y_valid_value.values)

In [176]:
predcit =pd.DataFrame(data,index=full_X.iloc[-14:,].index,columns=y_train.columns)

In [178]:
predcit.to_excel('predict.xls')